# Imposto de Renda

In [ ]:
ano = '2020'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widget

In [ ]:
sns.set(style="white")

plt.rcParams['axes.titlesize'] = 20
plt.rcParams['axes.labelsize'] = 20
plt.rcParams['axes.linewidth'] = 3
plt.rcParams['axes.labelpad'] = 15

plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['xtick.major.size'] = 8
plt.rcParams['xtick.major.width'] = 3

plt.rcParams['ytick.labelsize'] = 16
plt.rcParams['ytick.major.size'] = 8
plt.rcParams['ytick.major.width'] = 3

plt.rcParams['legend.fontsize'] = 14
plt.rcParams['legend.markerscale'] = 1

plt.rcParams['lines.markersize'] = 9
plt.rcParams['lines.linewidth'] = 2

### Lê os datasets

In [ ]:
df_ativos = pd.read_csv('Dataset_ações_' + ano + '.csv')
df_ativos['Data'] = pd.to_datetime(df_ativos['Data'], format="%Y/%m/%d")
df_ativos.head()

In [ ]:
df_proventos = pd.read_excel('Proventos - ' + ano + '.xlsx')
df_proventos.head()

# Funções

In [ ]:
def carteira_de_acoes_atual(df, offline = False):
    
    carteira = pd.DataFrame(columns=df.columns)

    cods = df['Código'].unique()
    
    for cod in cods:
        
        last_idx = df[df[['Código']] == cod].last_valid_index() 
        
        carteira = carteira.append(df.loc[last_idx], ignore_index=True)

    carteira = carteira[carteira['Quantidade Atual'] != 0.0]
    
    carteira.sort_values(by='Data', ascending=True, inplace=True)
    
    drop_columns = ['Especificação', 'Tipo de Execução', 'Quantidade de C/V', 'Mercado', 'Preço de C/V']
    
    carteira.drop(drop_columns, inplace=True, axis=1)
    
    carteira.rename(columns={'Preço Atual': '<Preço>'}, inplace=True)
    
    carteira.reset_index(drop=True, inplace=True)
    
    carteira['<Valor Total>'] = carteira['Quantidade Atual'] * carteira['<Preço>']
    
    if offline == False:
    
        carteira['Preço Atual'] = get_market_values(carteira['Código'])

        carteira['Valor Total Atual'] = carteira['Quantidade Atual'] * carteira['Preço Atual']

        carteira['L/P'] = carteira['Quantidade Atual'] * ( carteira['Preço Atual'] - carteira['<Preço>'] )

        carteira['L/P (%)'] = (carteira['L/P'] / (carteira['<Preço>'] * carteira['Quantidade Atual'])) * 100.0

        carteira['L/P'] = carteira['L/P'].round(2)

        carteira['L/P (%)'] = carteira['L/P (%)'].round(2) 

        carteira['<Preço>'] = carteira['<Preço>'].round(2)


        cols_ordenadas = ['Data', 'Código', 'Quantidade Atual', '<Preço>', '<Valor Total>',
                          'Preço Atual', 'Valor Total Atual', 'L/P', 'L/P (%)', 'Corretora']

    elif offline == True:
        
        cols_ordenadas = ['Data', 'Código', 'Quantidade Atual', '<Preço>', '<Valor Total>', 'Corretora']
        
    return carteira[cols_ordenadas]

# Vendas acima de 20k

- Meses com venda acima de 20k tem que pagar IR no próximo mês de 15% para swing trade.

In [ ]:
df_ativos['Mês'] = df_ativos['Data'].dt.strftime('%B')

In [ ]:
df_vendas = df_ativos.groupby(['Mês', 'Tipo de Execução'], sort=False).sum().reset_index()

In [ ]:
plt.figure(figsize=[15, 9])

palette_c = ['b' if val < 20000 else 'r' for val in df_vendas['Valor Total']]
more_20k = [ '< 20K' if val < 20000 else '> 20' for val in df_vendas['Valor Total'] ]

sns.barplot(data = df_vendas, x = 'Mês', hue=more_20k, y = 'Valor Total', palette=palette_c, ci=None)

plt.ylabel('Vendas (R$)')
plt.title('Valor total das vendas no mês')

plt.show()

# Declaração mensal

- Declarar prejuízos para abatimentos em cobranças posteriores.

In [ ]:
lucro_mes = df_ativos.loc[(df_ativos['L/P'] != 0.0), ['Data', 'L/P']]
lucro_mes['Mês'] = lucro_mes['Data'].dt.strftime('%b')
lucro_mes = lucro_mes.groupby(['Mês'], sort=False).sum().reset_index()

In [ ]:
plt.figure(figsize=[15, 9])

df = lucro_mes

palette_c = ['r' if c < 0.0 else 'b' for c in df['L/P']]

g = sns.barplot(data = df, x = 'Mês', y = 'L/P', palette=palette_c)

for index, row in df[['L/P']].reset_index().values:
    g.text(index, row, np.round(row, 2), color='black', ha="center", va='bottom',
           fontsize='large', fontweight='bold')
    
plt.hlines(y=0, xmax=df.shape[0], xmin=-1)
plt.ylabel('Lucro  (R$)')
plt.xlabel('')
plt.title('Lucro no ano')
plt.show()

In [ ]:
print('Soma em meses com lucro: {0}'.format(lucro_mes.loc[lucro_mes['L/P'] > 0, 'L/P'].sum().round(4)))
print('Soma em meses com prejuízo: {}'.format(lucro_mes.loc[lucro_mes['L/P'] < 0, 'L/P'].sum().round(4)))

# Proventos

In [ ]:
def f(tipo, previsão):
    df = df_proventos[df_proventos['Tipo'] == tipo]

    mask_year = df['Previsão'].dt.strftime('%Y')  == previsão
    mask_not_year = df['Previsão'].dt.strftime('%Y')  != previsão

    print('\nJá creditados em {0}:'.format(previsão))

    display(df[mask_year])

    print('\nCrédito em transito:')
    display(df[mask_not_year])

interactive(f, tipo=['jcp', 'div'] , previsão=[ano])

# Posições

### Zerada

In [ ]:
def f(corretora):

    df = df_ativos[df_ativos['Corretora'] == corretora]

    cods = df['Código'].unique()

    index = []

    for cod in cods:

        qnt = df.loc[df['Código'] == cod, 'Quantidade Atual'].tail(1)

        idx = qnt.index.values[0]

        if qnt.values[0] == 0.0:

            index.append(idx)

    df = df_ativos.iloc[index, :].copy()

    df.rename(columns={'Preço Atual': '<Preço>'}, inplace=True)
    
    df['<Valor Total>'] =  - df['Quantidade de C/V'] * df['<Preço>']
    
    df = df.sort_values('Data')
    
    df['Posição'] = 'Zerada'
    
    display(df[['Data', 'Código', 'Quantidade de C/V', '<Preço>', '<Valor Total>', 'Corretora', 'Posição']])

interactive(f, corretora=df_ativos['Corretora'].unique())

### Comprada

In [ ]:
def f(corretora):
    
    df = df_ativos[df_ativos['Corretora'] == corretora]
    df = carteira_de_acoes_atual(df.copy(), offline=True)
    df['Posição'] = 'Comprada'
    display(df)
    
interactive(f, corretora=df_ativos['Corretora'].unique())

# Movimentações por ativo

In [ ]:
def f(ativo):
    
    display(df_ativos[df_ativos['Código'] == ativo].drop('Mês', axis=1))
    
interactive(f, ativo=df_ativos['Código'].unique())

# Movimentação por corretora

In [ ]:
def f(corretora):
    
    display(df_ativos[df_ativos['Corretora'] == corretora].drop('Mês', axis=1))
    
interactive(f, corretora=df_ativos['Corretora'].unique())